In [ ]:
using DataStructures
using Dates

# Node types
abstract type AbstractNode end

mutable struct SourceNode{T} <: AbstractNode
    id::Symbol
    data::Vector{Tuple{DateTime, T}}  # (timestamp, value) pairs
    current_index::Int
    output::Union{T, Nothing}
end

mutable struct ComputeNode <: AbstractNode
    id::Symbol
    func::Function
    inputs::Vector{AbstractNode}
    output::Any
end

# Event type for simulation
struct Event
    timestamp::DateTime
    source_id::Symbol
end

# Define comparison for Event
Base.isless(a::Event, b::Event) = a.timestamp < b.timestamp

# Executor structure
mutable struct Executor
    nodes::Dict{Symbol, AbstractNode}
    dependencies::Dict{Symbol, Set{Symbol}}
    reverse_dependencies::Dict{Symbol, Set{Symbol}}
    topological_order::Vector{Symbol}
    simulation_heap::BinaryMinHeap{Event}
    current_time::DateTime
end

function Executor()
    executor = Executor(
        Dict{Symbol, AbstractNode}(),
        Dict{Symbol, Set{Symbol}}(),
        Dict{Symbol, Set{Symbol}}(),
        Symbol[],
        BinaryMinHeap{Event}(),
        DateTime(2000, 1, 1)  # Default start time
    )
    return executor
end

# Add a node to the executor using multiple dispatch
function add_node!(executor::Executor, node::SourceNode)
    executor.nodes[node.id] = node
    executor.dependencies[node.id] = Set{Symbol}()
    executor.reverse_dependencies[node.id] = Set{Symbol}()
    if !isempty(node.data)
        push!(executor.simulation_heap, Event(node.data[1][1], node.id))
    end
end

function add_node!(executor::Executor, node::ComputeNode)
    executor.nodes[node.id] = node
    executor.dependencies[node.id] = Set{Symbol}()
    executor.reverse_dependencies[node.id] = Set{Symbol}()
    for input in node.inputs
        push!(executor.dependencies[node.id], input.id)
        push!(get!(executor.reverse_dependencies, input.id, Set{Symbol}()), node.id)
    end
end

# Link nodes
function link_nodes!(executor::Executor, source::AbstractNode, target::ComputeNode)
    push!(target.inputs, source)
    push!(executor.dependencies[target.id], source.id)
    push!(get!(executor.reverse_dependencies, source.id, Set{Symbol}()), target.id)
end

# Topological sort
function topological_sort!(executor::Executor)
    in_degree = Dict(node_id => length(deps) for (node_id, deps) in executor.dependencies)
    queue = Queue{Symbol}()
    
    # Enqueue nodes with in-degree 0
    for (node_id, degree) in in_degree
        if degree == 0
            enqueue!(queue, node_id)
        end
    end
    
    sorted_order = Symbol[]
    
    while !isempty(queue)
        node_id = dequeue!(queue)
        push!(sorted_order, node_id)
        
        for dependent_id in get(executor.reverse_dependencies, node_id, Set{Symbol}())
            in_degree[dependent_id] -= 1
            if in_degree[dependent_id] == 0
                enqueue!(queue, dependent_id)
            end
        end
    end
    
    if length(sorted_order) != length(executor.nodes)
        error("Graph has a cycle")
    end
    
    executor.topological_order = sorted_order
end

# Execute a single node using multiple dispatch
function execute_node!(executor::Executor, node::SourceNode)
    if node.current_index <= length(node.data)
        node.output = node.data[node.current_index][2]
        node.current_index += 1
        # Add next event if available
        if node.current_index <= length(node.data)
            push!(executor.simulation_heap, Event(node.data[node.current_index][1], node.id))
        end
    end
end

function execute_node!(executor::Executor, node::ComputeNode)
    input_values = [executor.nodes[input.id].output for input in node.inputs]
    node.output = node.func(executor.current_time, input_values...)
end

function execute_node!(executor::Executor, node_id::Symbol)
    execute_node!(executor, executor.nodes[node_id])
end

# Process the computation graph starting from a specific node
function process_graph!(executor::Executor, start_node_id::Symbol)
    queue = Queue{Symbol}()
    enqueue!(queue, start_node_id)
    processed = Set{Symbol}()

    while !isempty(queue)
        node_id = dequeue!(queue)
        node = executor.nodes[node_id]

        if node_id in processed
            continue
        end

        if isa(node, ComputeNode) && !all(executor.nodes[input.id].output !== nothing for input in node.inputs)
            continue
        end

        execute_node!(executor, node_id)
        push!(processed, node_id)

        for dependent_id in get(executor.reverse_dependencies, node_id, Set{Symbol}())
            enqueue!(queue, dependent_id)
        end
    end
end

# Run simulation
function run_simulation!(executor::Executor)
    while !isempty(executor.simulation_heap)
        event = pop!(executor.simulation_heap)
        executor.current_time = event.timestamp
        execute_node!(executor, event.source_id)
        process_graph!(executor, event.source_id)
    end
end

# Source node creation
function create_source(id::Symbol, data::Vector{Tuple{DateTime, T}}) where T
    SourceNode{T}(id, sort(data), 1, nothing)
end

# Example usage
function run_example()
    executor = Executor()

    # Create source nodes with sample data
    source1 = create_source(:source1, [
        (DateTime(2000, 1, 1, 0, 0, 1), 2),
        (DateTime(2000, 1, 1, 0, 0, 3), 4),
        (DateTime(2000, 1, 1, 0, 0, 5), 6)
    ])
    source2 = create_source(:source2, [
        (DateTime(2000, 1, 1, 0, 0, 2), 10),
        (DateTime(2000, 1, 1, 0, 0, 4), 20),
        (DateTime(2000, 1, 1, 0, 0, 6), 30)
    ])

    # Create compute nodes
    square = ComputeNode(:square, (time, x) -> x^2, AbstractNode[], nothing)
    divide_by_2 = ComputeNode(:divide_by_2, (time, x) -> x / 2, AbstractNode[], nothing)
    multiply_by_neg1 = ComputeNode(:multiply_by_neg1, (time, x) -> -x, AbstractNode[], nothing)
    combine = ComputeNode(:combine, (time, x, y) -> (x, y), AbstractNode[], nothing)
    final_multiply = ComputeNode(:final_multiply, (time, tuple, source2_val) -> (tuple[1] * source2_val, tuple[2] * source2_val), AbstractNode[], nothing)
    output = ComputeNode(:output, (time, x) -> println("Output at time $time: $x"), AbstractNode[], nothing)

    # Add nodes to executor
    add_node!(executor, source1)
    add_node!(executor, source2)
    add_node!(executor, square)
    add_node!(executor, divide_by_2)
    add_node!(executor, multiply_by_neg1)
    add_node!(executor, combine)
    add_node!(executor, final_multiply)
    add_node!(executor, output)

    # Link nodes
    link_nodes!(executor, source1, square)
    link_nodes!(executor, square, divide_by_2)
    link_nodes!(executor, source2, multiply_by_neg1)
    link_nodes!(executor, divide_by_2, combine)
    link_nodes!(executor, multiply_by_neg1, combine)
    link_nodes!(executor, combine, final_multiply)
    link_nodes!(executor, source2, final_multiply)
    link_nodes!(executor, final_multiply, output)

    # Perform topological sort
    topological_sort!(executor)

    # Run simulation
    run_simulation!(executor)
end

run_example()

In [ ]:
using Dates

executor = Executor()

# Create source nodes with sample data
source1 = create_source(:source1, [
    (DateTime(2000, 1, 1, 0, 0, 1), 2),
    (DateTime(2000, 1, 1, 0, 0, 3), 4),
    (DateTime(2000, 1, 1, 0, 0, 5), 6)
])
source2 = create_source(:source2, [
    (DateTime(2000, 1, 1, 0, 0, 2), 10),
    (DateTime(2000, 1, 1, 0, 0, 4), 20),
    (DateTime(2000, 1, 1, 0, 0, 6), 30)
])

# Create compute nodes
square = ComputeNode(:square, (time, x) -> x^2, AbstractNode[], nothing)
divide_by_2 = ComputeNode(:divide_by_2, (time, x) -> x / 2, AbstractNode[], nothing)
multiply_by_neg1 = ComputeNode(:multiply_by_neg1, (time, x) -> -x, AbstractNode[], nothing)
combine = ComputeNode(:combine, (time, x, y) -> (x, y), AbstractNode[], nothing)
final_multiply = ComputeNode(:final_multiply, (time, tuple, source2_val) -> (tuple[1] * source2_val, tuple[2] * source2_val), AbstractNode[], nothing)
output = ComputeNode(:output, (time, x) -> println("Output at time $time: $x"), AbstractNode[], nothing)

# Add nodes to executor
add_node!(executor, source1)
add_node!(executor, source2)
add_node!(executor, square)
add_node!(executor, divide_by_2)
add_node!(executor, multiply_by_neg1)
add_node!(executor, combine)
add_node!(executor, final_multiply)
add_node!(executor, output)

# Link nodes
link_nodes!(executor, source1, square)
link_nodes!(executor, square, divide_by_2)
link_nodes!(executor, source2, multiply_by_neg1)
link_nodes!(executor, divide_by_2, combine)
link_nodes!(executor, multiply_by_neg1, combine)
link_nodes!(executor, combine, final_multiply)
link_nodes!(executor, source2, final_multiply)
link_nodes!(executor, final_multiply, output)

# Perform topological sort
topological_sort!(executor)

# # Visualize the graph
# plt = visualize_graph(executor)
# display(plt)

# # Save the plot to a file
# savefig(plt, "computation_graph.png")
# println("Plot saved to computation_graph.png")

In [ ]:
function visualize_graph(graph::StreamGraph)
    nodes = graph.nodes
    g = SimpleDiGraph(length(nodes))
    nlabels = String[]
    node_colors = []
    
    # Create a mapping from node IDs to graph indices
    node_mapping = Dict(node_id => i for (i, node_id) in enumerate(keys(nodes)))
    zero_levels = Vector{Pair{Int, Int}}()
    nlabels_align = []
    nlabels_color = []

    for (i, (node_id, node)) in enumerate(nodes)
        push!(nlabels, string(node_id))
        if isa(node, SourceNode)
            push!(node_colors, colorant"#ffdd33")  # yellow for source nodes
            push!(zero_levels, i => 1) # source layers always at level 1
            push!(nlabels_align, (:center, :bottom))
            push!(nlabels_color, colorant"#bf489d")
        else
            push!(node_colors, colorant"#b4dee8")  # light blue for compute nodes
            push!(nlabels_align, (:center, :top))
            push!(nlabels_color, colorant"#000000")
        end
        
        for dep_id in graph.dependencies[node_id]
            add_edge!(g, node_mapping[dep_id], node_mapping[node_id])
        end
    end

    xs, ys, paths = solve_positions(Zarate(), g; force_layer=zero_levels)
    xs, ys = -ys, -xs # rotate coordinates by 90°
    ys .*= 0.5 # scale the y coordinates
    lay = Point.(zip(xs, ys))

    f, ax, p = graphplot(g;
        layout=lay,
        arrow_size=15,
        arrow_shift=:end,
        arrow_marker='>',
        edge_width=0.75,
        edge_color=colorant"#444",
        # node_color=node_colors,
        # node_size=48,
        nlabels=nlabels,
        nlabels_fontsize=14,
        nlabels_align=nlabels_align,
        nlabels_color=nlabels_color,
        # nlabels_distance=12,
        node_size=40,
        node_color=:white,
        nlabels_distance=-10,
    )
    hidedecorations!(ax)
    hidespines!(ax)

    # add some padding
    x_range, y_range = extrema(xs), extrema(ys)
    x_range, y_range = x_range[2] - x_range[1], y_range[2] - y_range[1]
    xlims!(ax, minimum(xs) - 0.3x_range, maximum(xs) + 0.3x_range)
    ylims!(ax, minimum(ys) - 0.1y_range, maximum(ys) + 0.1y_range)

    # adjust width to match aspect ratio
    resize!(f, floor(Int, size(f.scene)[2]*(x_range / y_range)), size(f.scene)[2])
    
    f, ax, p
end

f, ax, p = visualize_graph(graph)

# Save the plot to a file
# save("computation_graph.png", f)

display(f);

remove data and current_index from StreamNode

In [3]:
using DataStructures
using Dates

# Unified StreamNode struct with last_value field
mutable struct StreamNode
    id::Symbol
    func::Function
    inputs::Vector{Int}
    output::Any
    last_value::Any
    index::Int
end

# Helper function to check if a node is a source node
@inline is_source(node::StreamNode) = isempty(node.inputs)

# Event type for simulation
struct Event
    timestamp::DateTime
    source_index::Int
end

# Define comparison for Event
Base.isless(a::Event, b::Event) = a.timestamp < b.timestamp

# StreamGraph structure
mutable struct StreamGraph
    nodes::Vector{StreamNode} # list of nodes
    dependencies::Vector{Vector{Int}} # adjacency list
    reverse_dependencies::Vector{Vector{Int}} # reverse adjacency list
    topological_order::Vector{Int} # list of node indices in topological order
end

function StreamGraph()
    StreamGraph(
        StreamNode[],
        Vector{Int}[],
        Vector{Int}[],
        Int[]
    )
end

# Executor structure
mutable struct Executor
    graph::StreamGraph
    simulation_heap::BinaryMinHeap{Event}
    current_time::DateTime
end

function Executor(graph::StreamGraph)
    Executor(
        graph,
        BinaryMinHeap{Event}(),
        DateTime(2000, 1, 1)  # Default start time
    )
end

# Add a source node to the graph
function create_source!(graph::StreamGraph, id::Symbol, data::Vector{Tuple{DateTime, T}}) where T
    index = length(graph.nodes) + 1
    current_index = Ref(1)

    function source_func(time::DateTime, executor::Executor)
        if current_index[] <= length(data)
            timestamp, output = data[current_index[]]
            current_index[] += 1
            # Add next event if available
            if current_index[] <= length(data)
                next_timestamp, _ = data[current_index[]]
                push!(executor.simulation_heap, Event(next_timestamp, index))
            end
            return output
        end
        nothing
    end

    node = StreamNode(id, source_func, Int[], nothing, nothing, index)
    push!(graph.nodes, node)
    push!(graph.dependencies, Int[])
    push!(graph.reverse_dependencies, Int[])
    index
end

# Add a compute node to the graph
function create_compute!(graph::StreamGraph, id::Symbol, func::Function, input_indices::Vector{Int})
    index = length(graph.nodes) + 1
    node = StreamNode(id, func, input_indices, nothing, nothing, index)
    push!(graph.nodes, node)
    push!(graph.dependencies, copy(input_indices))
    push!(graph.reverse_dependencies, Int[])
    for input_index in input_indices
        push!(graph.reverse_dependencies[input_index], index)
    end
    index
end

# Topological sort
function topological_sort!(graph::StreamGraph)
    in_degree = [length(deps) for deps in graph.dependencies]
    queue = Int[]
    
    # Add nodes with in-degree 0
    for (index, degree) in enumerate(in_degree)
        if degree == 0
            push!(queue, index)
        end
    end
    
    sorted_order = Int[]
    
    while !isempty(queue)
        node_index = pop!(queue)
        push!(sorted_order, node_index)
        
        for dependent_index in graph.reverse_dependencies[node_index]
            in_degree[dependent_index] -= 1
            if in_degree[dependent_index] == 0
                push!(queue, dependent_index)
            end
        end
    end
    
    if length(sorted_order) != length(graph.nodes)
        error("Graph has a cycle")
    end
    
    graph.topological_order = sorted_order
    nothing
end

# Execute a single node
function execute_node!(executor::Executor, node::StreamNode, is_event_source::Bool)
    if is_source(node)
        # only execute source function if it is the trigger for the current event
        node.output = is_event_source ? node.func(executor.current_time, executor) : node.last_value
    else
        input_values = (executor.graph.nodes[input_index].output for input_index in node.inputs)
        node.output = any(isnothing, input_values) ? nothing : node.func(executor.current_time, input_values...)
    end
    
    node.last_value = node.output

    nothing
end

# Process the relevant subgraph starting from a specific node
function process_subgraph!(executor::Executor, start_node_index::Int)
    queue = PriorityQueue{Int, Int}()
    enqueue!(queue, start_node_index => 0)
    depths = Dict(start_node_index => 0)

    while !isempty(queue)
        node_index = dequeue!(queue)
        is_event_source = (node_index == start_node_index)
        execute_node!(executor, executor.graph.nodes[node_index], is_event_source)

        current_depth = depths[node_index]
        for dependent_index in executor.graph.reverse_dependencies[node_index]
            new_depth = current_depth + 1
            if get(depths, dependent_index, -1) < new_depth
                depths[dependent_index] = new_depth
                if haskey(queue, dependent_index)
                    queue[dependent_index] = -new_depth  # Update priority if already in queue
                else
                    enqueue!(queue, dependent_index => -new_depth)
                end
            end
        end
    end

    nothing
end

# Run simulation
function run_simulation!(executor::Executor)
    # Initialize simulation heap with initial events
    for node in graph.nodes
        if is_source(node)
            execute_node!(executor, node, true)
        end
    end

    while !isempty(executor.simulation_heap)
        event = pop!(executor.simulation_heap)
        executor.current_time = event.timestamp
        process_subgraph!(executor, event.source_index)
    end

    nothing
end

# Create a new graph
graph = StreamGraph()

# Create source nodes with sample data
source1_index = create_source!(graph, :source1, [
    (DateTime(2000, 1, 1, 0, 0, 1), 2),
    (DateTime(2000, 1, 1, 0, 0, 3), 4),
    (DateTime(2000, 1, 1, 0, 0, 5), 6)
])
source2_index = create_source!(graph, :source2, [
    (DateTime(2000, 1, 1, 0, 0, 2), 10),
    (DateTime(2000, 1, 1, 0, 0, 4), 20),
    (DateTime(2000, 1, 1, 0, 0, 6), 30)
])

# Create compute nodes
square_index = create_compute!(graph, :square, (time, x) -> x^2, [source1_index])
divide_by_2_index = create_compute!(graph, :divide_by_2, (time, x) -> x / 2, [square_index])
multiply_by_neg1_index = create_compute!(graph, :multiply_by_neg1, (time, x) -> -x, [source2_index])
combine_index = create_compute!(graph, :combine, (time, x, y) -> (x, y), [divide_by_2_index, multiply_by_neg1_index])
final_multiply_index = create_compute!(graph, :final_multiply, (time, tuple, source2_val) -> (tuple[1] * source2_val, tuple[2] * source2_val), [combine_index, source2_index])
output_index = create_compute!(graph, :output, (time, x) -> println("Final Output at time $time: $x"), [final_multiply_index])

# Perform topological sort
topological_sort!(graph)

# Create executor
executor = Executor(graph)

# Run simulation
run_simulation!(executor)

Final Output at time 2000-01-01T00:00:04: (160.0, -400)
Final Output at time 2000-01-01T00:00:05: (360.0, -400)
Final Output at time 2000-01-01T00:00:06: (540.0, -900)


bind_inputs!